# ***Step 1: Import libraries & Set up***

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 12 # N_mffc
hidden_size = 64
num_layers = 3
num_classes = 35
batch_size = 1
num_epochs = 25
learning_rate = 0.001

# ***Step 2: Use trained model (FP32)***
Reference: https://github.com/felixchenfy/Speech-Commands-Classification-by-LSTM-PyTorch

In [ ]:
# Recurrent neural network (many-to-one)
class LSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        return x
model = LSTM(12, 35,64,3)
x = torch.randn(batch_size, sequence_length, input_size)
out = model(x)
print(out)



tensor([[[-2.5290e-01, -1.8758e+00,  1.3415e+00, -1.1997e+00,  2.1117e+00,
          -3.9076e-02,  6.1993e-02,  4.6316e-01,  1.1036e+00, -9.3451e-01,
          -5.0276e-01, -1.2987e+00],
         [-3.0322e-01,  3.4131e-02,  1.2128e+00,  6.8188e-01,  4.9008e-01,
          -4.7426e-01, -6.3215e-01,  1.1314e+00,  3.0332e+00, -2.3666e-01,
           7.4033e-01, -6.8676e-01],
         [ 4.3509e-01,  4.3338e-01,  9.9051e-02,  1.3114e+00,  3.0166e-01,
           7.0157e-01, -6.0318e-01,  1.1922e-01, -1.4153e+00,  6.0212e-01,
           1.1867e+00,  5.9761e-01],
         [ 3.9018e-02, -7.1948e-02, -2.9730e+00,  2.9833e-01,  9.6815e-01,
          -6.8615e-01,  4.7431e-01, -1.1082e-01,  1.0843e+00, -5.8207e-02,
          -1.1456e+00,  7.1485e-01],
         [ 4.1550e-01,  4.3655e-01, -7.1200e-02,  1.9184e+00,  1.3716e+00,
           7.7067e-02,  6.9612e-01, -3.4883e-01,  8.2862e-02, -8.6443e-01,
          -1.1039e+00, -8.0562e-01],
         [-9.0392e-01,  3.1902e-01,  8.6414e-01,  1.5079e+00,  6.

***--- Above LSTM model tensors representation for Affine per tensor quantization--***

In [ ]:
import os

In [ ]:
T = torch.tensor(out)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


***--Maximum Value and minimum value of x***

In [ ]:
# Min-Max value of float_32 tensor (x) find out for scale (s) and zero point (z)

b = torch.max(T)
print(b)

tensor(3.0332)


In [ ]:
a = torch.min(T)
print(a)

tensor(-2.9730)


In [ ]:
# scale value

s = (b-a)/16777215

print(s)

tensor(3.5800e-07)


In [ ]:
# zero point

z = torch.round(-a*16777215/(b-a))

print(z)

tensor(8304584.)


# ***Step-3: Apply Quantization by round and clipping function (Affine mapping)***

In [ ]:
f = torch.round(T/s + z)
print(f)
Tq = torch.clip(f, min=0, max=16777215) # Here min & max value we can change as per Tbit.
# But, I have checked for 24 bit
print (Tq)
torch.save(Tq,'qtz_tensor.pt')


tensor([[[ 7598148.,  3064866., 12051838.,  4953568., 14203323.,  8195432.,
           8477751.,  9598345., 11387341.,  5694198.,  6900208.,  4676987.],
         [ 7457608.,  8399923., 11692388., 10209300.,  9673544.,  6979822.,
           6538782., 11464866., 16777215.,  7643512., 10372557.,  6386241.],
         [ 9519915.,  9515156.,  8581263., 11967839.,  9147214., 10264299.,
           6619702.,  8637610.,  4351332.,  9986501., 11619276.,  9973911.],
         [ 8413574.,  8103610.,        0.,  9137902., 11008938.,  6387942.,
           9629475.,  7995025., 11333414.,  8141994.,  5104546., 10301388.],
         [ 9465195.,  9523996.,  8105700., 13663266., 12135799.,  8519857.,
          10249067.,  7330200.,  8536043.,  5889950.,  5221109.,  6054231.],
         [ 5779650.,  9195707., 10718395., 12516616., 10048631.,  7960708.,
           7847688.,  3455976.,  7840659.,  6657510.,  6960778.,  8024534.],
         [ 3571385.,  9437158.,  8666212.,  6356301., 12389837.,  2283570.,
      

# ***Step-4: Apply Dequantization***

In [ ]:
Tdq = s*(Tq - z)
print(Tdq)

tensor([[[-2.5290e-01, -1.8758e+00,  1.3415e+00, -1.1997e+00,  2.1117e+00,
          -3.9076e-02,  6.1993e-02,  4.6316e-01,  1.1036e+00, -9.3451e-01,
          -5.0276e-01, -1.2987e+00],
         [-3.0322e-01,  3.4131e-02,  1.2128e+00,  6.8188e-01,  4.9008e-01,
          -4.7426e-01, -6.3215e-01,  1.1314e+00,  3.0332e+00, -2.3666e-01,
           7.4033e-01, -6.8676e-01],
         [ 4.3509e-01,  4.3338e-01,  9.9050e-02,  1.3114e+00,  3.0166e-01,
           7.0157e-01, -6.0318e-01,  1.1922e-01, -1.4153e+00,  6.0212e-01,
           1.1867e+00,  5.9761e-01],
         [ 3.9018e-02, -7.1948e-02, -2.9730e+00,  2.9833e-01,  9.6815e-01,
          -6.8615e-01,  4.7431e-01, -1.1082e-01,  1.0843e+00, -5.8207e-02,
          -1.1456e+00,  7.1485e-01],
         [ 4.1550e-01,  4.3655e-01, -7.1200e-02,  1.9184e+00,  1.3716e+00,
           7.7067e-02,  6.9612e-01, -3.4883e-01,  8.2862e-02, -8.6443e-01,
          -1.1039e+00, -8.0562e-01],
         [-9.0392e-01,  3.1902e-01,  8.6414e-01,  1.5079e+00,  6.

# ***Step-5: MAE/MSE loss between T and Tdq***

***I. MAE loss***

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn
# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq)
# create a criterion to measure the mean absolute error
mae = nn.L1Loss()
# compute the loss (mean absolute error)
output = mae(T, Tdq)
# output.backward()
print("MAE loss:", output)

Input Tensor:
 tensor([[[-2.5290e-01, -1.8758e+00,  1.3415e+00, -1.1997e+00,  2.1117e+00,
          -3.9076e-02,  6.1993e-02,  4.6316e-01,  1.1036e+00, -9.3451e-01,
          -5.0276e-01, -1.2987e+00],
         [-3.0322e-01,  3.4131e-02,  1.2128e+00,  6.8188e-01,  4.9008e-01,
          -4.7426e-01, -6.3215e-01,  1.1314e+00,  3.0332e+00, -2.3666e-01,
           7.4033e-01, -6.8676e-01],
         [ 4.3509e-01,  4.3338e-01,  9.9051e-02,  1.3114e+00,  3.0166e-01,
           7.0157e-01, -6.0318e-01,  1.1922e-01, -1.4153e+00,  6.0212e-01,
           1.1867e+00,  5.9761e-01],
         [ 3.9018e-02, -7.1948e-02, -2.9730e+00,  2.9833e-01,  9.6815e-01,
          -6.8615e-01,  4.7431e-01, -1.1082e-01,  1.0843e+00, -5.8207e-02,
          -1.1456e+00,  7.1485e-01],
         [ 4.1550e-01,  4.3655e-01, -7.1200e-02,  1.9184e+00,  1.3716e+00,
           7.7067e-02,  6.9612e-01, -3.4883e-01,  8.2862e-02, -8.6443e-01,
          -1.1039e+00, -8.0562e-01],
         [-9.0392e-01,  3.1902e-01,  8.6414e-01,  

***II. MSE Loss***

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn

# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq)

# create a criterion to measure the mean squared error
mse = nn.MSELoss()

# compute the loss (mean squared error)
output = mse(T, Tdq)

# output.backward()
print("MSE loss:", output)

Input Tensor:
 tensor([[[-2.5290e-01, -1.8758e+00,  1.3415e+00, -1.1997e+00,  2.1117e+00,
          -3.9076e-02,  6.1993e-02,  4.6316e-01,  1.1036e+00, -9.3451e-01,
          -5.0276e-01, -1.2987e+00],
         [-3.0322e-01,  3.4131e-02,  1.2128e+00,  6.8188e-01,  4.9008e-01,
          -4.7426e-01, -6.3215e-01,  1.1314e+00,  3.0332e+00, -2.3666e-01,
           7.4033e-01, -6.8676e-01],
         [ 4.3509e-01,  4.3338e-01,  9.9051e-02,  1.3114e+00,  3.0166e-01,
           7.0157e-01, -6.0318e-01,  1.1922e-01, -1.4153e+00,  6.0212e-01,
           1.1867e+00,  5.9761e-01],
         [ 3.9018e-02, -7.1948e-02, -2.9730e+00,  2.9833e-01,  9.6815e-01,
          -6.8615e-01,  4.7431e-01, -1.1082e-01,  1.0843e+00, -5.8207e-02,
          -1.1456e+00,  7.1485e-01],
         [ 4.1550e-01,  4.3655e-01, -7.1200e-02,  1.9184e+00,  1.3716e+00,
           7.7067e-02,  6.9612e-01, -3.4883e-01,  8.2862e-02, -8.6443e-01,
          -1.1039e+00, -8.0562e-01],
         [-9.0392e-01,  3.1902e-01,  8.6414e-01,  

# ***Method-2:- Symmetric quantization scheme***

In [ ]:
f = torch.round(T/s + z)
print(f)
Tq1 = torch.clip(f, min=-8388608, max=8388608) # Here min & max value we can change as per Tbit.
# But, I have checked for 24 bit
print (Tq1)
torch.save(Tq1,'qtz_tensor.pt')


tensor([[[ 7598148.,  3064866., 12051838.,  4953568., 14203323.,  8195432.,
           8477751.,  9598345., 11387341.,  5694198.,  6900208.,  4676987.],
         [ 7457608.,  8399923., 11692388., 10209300.,  9673544.,  6979822.,
           6538782., 11464866., 16777215.,  7643512., 10372557.,  6386241.],
         [ 9519915.,  9515156.,  8581263., 11967839.,  9147214., 10264299.,
           6619702.,  8637610.,  4351332.,  9986501., 11619276.,  9973911.],
         [ 8413574.,  8103610.,        0.,  9137902., 11008938.,  6387942.,
           9629475.,  7995025., 11333414.,  8141994.,  5104546., 10301388.],
         [ 9465195.,  9523996.,  8105700., 13663266., 12135799.,  8519857.,
          10249067.,  7330200.,  8536043.,  5889950.,  5221109.,  6054231.],
         [ 5779650.,  9195707., 10718395., 12516616., 10048631.,  7960708.,
           7847688.,  3455976.,  7840659.,  6657510.,  6960778.,  8024534.],
         [ 3571385.,  9437158.,  8666212.,  6356301., 12389837.,  2283570.,
      

In [ ]:
Tdq1 = s*(Tq1 - z)
print(Tdq1)

tensor([[[-2.5290e-01, -1.8758e+00,  3.0080e-02, -1.1997e+00,  3.0080e-02,
          -3.9076e-02,  3.0080e-02,  3.0080e-02,  3.0080e-02, -9.3451e-01,
          -5.0276e-01, -1.2987e+00],
         [-3.0322e-01,  3.0080e-02,  3.0080e-02,  3.0080e-02,  3.0080e-02,
          -4.7426e-01, -6.3215e-01,  3.0080e-02,  3.0080e-02, -2.3666e-01,
           3.0080e-02, -6.8676e-01],
         [ 3.0080e-02,  3.0080e-02,  3.0080e-02,  3.0080e-02,  3.0080e-02,
           3.0080e-02, -6.0318e-01,  3.0080e-02, -1.4153e+00,  3.0080e-02,
           3.0080e-02,  3.0080e-02],
         [ 3.0080e-02, -7.1948e-02, -2.9730e+00,  3.0080e-02,  3.0080e-02,
          -6.8615e-01,  3.0080e-02, -1.1082e-01,  3.0080e-02, -5.8207e-02,
          -1.1456e+00,  3.0080e-02],
         [ 3.0080e-02,  3.0080e-02, -7.1200e-02,  3.0080e-02,  3.0080e-02,
           3.0080e-02,  3.0080e-02, -3.4883e-01,  3.0080e-02, -8.6443e-01,
          -1.1039e+00, -8.0562e-01],
         [-9.0392e-01,  3.0080e-02,  3.0080e-02,  3.0080e-02,  3.

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn
# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq1)
# create a criterion to measure the mean absolute error
mae = nn.L1Loss()
# compute the loss (mean absolute error)
output = mae(T, Tdq1)
# output.backward()
print("MAE loss:", output)

Input Tensor:
 tensor([[[-2.5290e-01, -1.8758e+00,  1.3415e+00, -1.1997e+00,  2.1117e+00,
          -3.9076e-02,  6.1993e-02,  4.6316e-01,  1.1036e+00, -9.3451e-01,
          -5.0276e-01, -1.2987e+00],
         [-3.0322e-01,  3.4131e-02,  1.2128e+00,  6.8188e-01,  4.9008e-01,
          -4.7426e-01, -6.3215e-01,  1.1314e+00,  3.0332e+00, -2.3666e-01,
           7.4033e-01, -6.8676e-01],
         [ 4.3509e-01,  4.3338e-01,  9.9051e-02,  1.3114e+00,  3.0166e-01,
           7.0157e-01, -6.0318e-01,  1.1922e-01, -1.4153e+00,  6.0212e-01,
           1.1867e+00,  5.9761e-01],
         [ 3.9018e-02, -7.1948e-02, -2.9730e+00,  2.9833e-01,  9.6815e-01,
          -6.8615e-01,  4.7431e-01, -1.1082e-01,  1.0843e+00, -5.8207e-02,
          -1.1456e+00,  7.1485e-01],
         [ 4.1550e-01,  4.3655e-01, -7.1200e-02,  1.9184e+00,  1.3716e+00,
           7.7067e-02,  6.9612e-01, -3.4883e-01,  8.2862e-02, -8.6443e-01,
          -1.1039e+00, -8.0562e-01],
         [-9.0392e-01,  3.1902e-01,  8.6414e-01,  

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn

# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq1)

# create a criterion to measure the mean squared error
mse = nn.MSELoss()

# compute the loss (mean squared error)
output = mse(T, Tdq1)

# output.backward()
print("MSE loss:", output)

Input Tensor:
 tensor([[[-2.5290e-01, -1.8758e+00,  1.3415e+00, -1.1997e+00,  2.1117e+00,
          -3.9076e-02,  6.1993e-02,  4.6316e-01,  1.1036e+00, -9.3451e-01,
          -5.0276e-01, -1.2987e+00],
         [-3.0322e-01,  3.4131e-02,  1.2128e+00,  6.8188e-01,  4.9008e-01,
          -4.7426e-01, -6.3215e-01,  1.1314e+00,  3.0332e+00, -2.3666e-01,
           7.4033e-01, -6.8676e-01],
         [ 4.3509e-01,  4.3338e-01,  9.9051e-02,  1.3114e+00,  3.0166e-01,
           7.0157e-01, -6.0318e-01,  1.1922e-01, -1.4153e+00,  6.0212e-01,
           1.1867e+00,  5.9761e-01],
         [ 3.9018e-02, -7.1948e-02, -2.9730e+00,  2.9833e-01,  9.6815e-01,
          -6.8615e-01,  4.7431e-01, -1.1082e-01,  1.0843e+00, -5.8207e-02,
          -1.1456e+00,  7.1485e-01],
         [ 4.1550e-01,  4.3655e-01, -7.1200e-02,  1.9184e+00,  1.3716e+00,
           7.7067e-02,  6.9612e-01, -3.4883e-01,  8.2862e-02, -8.6443e-01,
          -1.1039e+00, -8.0562e-01],
         [-9.0392e-01,  3.1902e-01,  8.6414e-01,  